In [1]:
import Ipynb_importer
from z_layers import *

importing Jupyter notebook from z_layers.ipynb


In [1]:
from functools import wraps

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Add, BatchNormalization, Concatenate,
                                     Conv2D, Layer, LeakyReLU, MaxPooling2D,
                                     UpSampling2D, ZeroPadding2D)
from tensorflow.keras.regularizers import l2

网络架构图：

<img src="https://pic3.zhimg.com/80/v2-139a50003c09efe54b2db906710f6252_1440w.jpg" alt="img" style="zoom:30%;" />

基本组件：
1. **CBM：**Yolov4网络结构中的最小组件，由Conv+Bn+Mish激活函数三者组成。
2. **CBL：**由Conv+Bn+Leaky_relu激活函数三者组成。
3. **Res unit：**借鉴Resnet网络中的残差结构，让网络可以构建的更深。
4. **CSPX：**借鉴CSPNet网络结构，由卷积层和X个Res unint模块Concate组成。


## 1、定义 csp_resblock_body
CSP 基本结构见上图

In [2]:
def csp_resblock_body(x, num_filters, num_blocks, all_narrow=True):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # 填充x的边界为0，由(?, 416, 416, 32)转换为(?, 417, 417, 32)。
    # 因为下一步卷积操作的步长为2，所以图的边长需要是奇数。
    x = ZeroPadding2D(((1, 0), (1, 0)))(x)
    # 第一个CBM对高和宽进行压缩
    x = Darknet_CBM(num_filters, (3,3), strides=(2,2))(x)  # yoloz中只有卷积层，通过调节卷积步长控制输出特征图的尺寸

    # 残差
    res_connection = Darknet_CBM(num_filters//2 if all_narrow else num_filters, (1,1))(x)
    # 主干
    x = Darknet_CBM(num_filters//2 if all_narrow else num_filters, (1,1))(x)
    for i in range(num_blocks):
        x_blocks = compose(
                Darknet_CBM(num_filters//2, (1,1)),
                Darknet_CBM(num_filters//2 if all_narrow else num_filters, (3,3)))(x)
        x = Add()([x, x_blocks])
        
    x = Darknet_CBM(num_filters//2 if all_narrow else num_filters, (1,1))(x)
    x = Concatenate()([x, res_connection])  # 主干、残差汇合
    x = Darknet_CBM(num_filters,(1,1))(x)
    
    return x

## 2、定义 csp_darknet53_body

In [3]:
def csp_darknet53_body(x):
    '''CSPDarknet53 body having 52 Convolution2D layers'''
    x = Darknet_CBM(32, (3,3))(x)
    x = csp_resblock_body(x, 64, 1, False)
    x = csp_resblock_body(x, 128, 2)
    x = csp_resblock_body(x, 256, 8)
    x = csp_resblock_body(x, 512, 8)
    x = csp_resblock_body(x, 1024, 4)
    return x